# Quarterly Sales Tax Revenue Analysis

### [Data Source](https://tax.utah.gov/econstats/sales)

### Q3: January, February, March

In [1]:
import pandas as pd
import os
import numpy as np
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Removes misc characters from string
def replace(string):
    string = string.replace("&", "")
    string = string.replace("-", " ")
    string = string.replace("(", "")
    string = string.replace(")", "")
    string = string.replace("/", " ")
    string = string.replace(",", "")
    string = string.replace(".", "")
    string = string.replace(" ", "_")
    return string


### Load Q3 2020 sales tax revenue data

In [4]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2020-03-quarterly-sales.xlsx'
q3_2020 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q3_2020.drop(q3_2020.tail(2).index,inplace=True)

# forward fill values from merged cells
q3_2020['County'] = q3_2020['County'].fillna(method='ffill')
q3_2020['Location Code'] = q3_2020['Location Code'].fillna(method='ffill')
q3_2020['City'] = q3_2020['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q3_2020.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q3_2020['Rounded Up'] = q3_2020['Rounded Up'].map(crosswalk)

q3_2020.head()


,County,Location Code,City,Economic Sector (NAICS Code),2020Q3,Rounded Up
0,Beaver County,1002.0,Beaver City,"AGRICULTURE, FORESTRY, FISHING & HUNTING (11)",80000.0,True
1,Beaver County,1002.0,Beaver City,UTILITIES (22),700000.0,True
2,Beaver County,1002.0,Beaver City,CONSTRUCTION (23),150000.0,True
3,Beaver County,1002.0,Beaver City,MANUFACTURING (31-33),1880967.0,False
4,Beaver County,1002.0,Beaver City,WHOLESALE TRADE-DURABLE GOODS (423),310250.0,False


In [5]:
# pivot the table to get sectors as columns with tax values by city
sectors_q3_2020 = pd.pivot_table(q3_2020,values = '2020Q3',index ='City', columns = 'Economic Sector (NAICS Code)', aggfunc='first')

# fill NAs with 0
sectors_q3_2020 = sectors_q3_2020.fillna(0)


# fix column names
#new_names = [replace(item) for item in list(sectors_q3_2020.columns)]
new_names = ['L_ACCOMMODATION_20', 'S_ADMIN_SUPPORT_20','A_AG_WILDLIFE_20','L_CULTURAL_REC_20','C_CONSTRUCTION_20',
'E_EDUCATION_20','S_FINANCIAL_20','L_RSTRNT_BAR_20','H_HEALTH_CARE_20','S_IT_DATA_20','S_CORPORATE_MGMT_20',
'M_MANUFACTURING_20','E_EXTRACTION_20','O_OTHER_20','X_ADJUSTMENTS_20','R_AUTO_PRIVATE_20','S_PROF_TECH_SRV_20',
'G_GOVERNMENT_20','S_RENTAL_LEASING_20','R_BUILDING_SUPPLY_20','R_CLOTHING_20','R_ELECTRONICS_20','R_GROCERY_BEV_20',
'R_FURNITURE_20','R_GAS_STATIONS_20','R_GENERAL_RETAIL_20','R_HEALTH_RETAIL_20','R_OTHER_RETAIL_20','R_AUTO_RETAIL_20',
'R_NONSTORE_RETAIL_20','R_SPORT_HOBBY_20','L_SPECIAL_EVENT_20','W_DISTRIBUTION_20','X_UNKNOWN_20','U_UTILITIES_20',
'W_WHLSALE_DURABLE_20','S_WHLSLE_ETRADE_20','W_WHLSLE_NDURABLE_20']
sectors_q3_2020.columns = new_names

# get the total sales tax revenue
sectors_q3_2020['TOTAL_20'] = sectors_q3_2020[new_names].sum(axis=1)

sectors_q3_2020.head()

ValueError: Length mismatch: Expected axis has 37 elements, new values have 38 elements

### Load Q3 2019 sales tax revenue data

In [ ]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2019-01-quarterly-sales.xlsx'
q3_2019 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q3_2019.drop(q3_2019.tail(2).index,inplace=True)

# forward fill values from merged cells
q3_2019['County'] = q3_2019['County'].fillna(method='ffill')
q3_2019['Location Code'] = q3_2019['Location Code'].fillna(method='ffill')
q3_2019['City'] = q3_2019['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q3_2019.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q3_2019['Rounded Up'] = q3_2019['Rounded Up'].map(crosswalk)

q3_2019.tail()

In [ ]:
# pivot the table to get sectors as columns with tax values by city
sectors_q3_2019 = pd.pivot_table(q3_2019,values = '2019Q3',index ='City', columns = 'Economic Sector (NAICS Code)', aggfunc='first')

# fill NAs with 0
sectors_q3_2019 = sectors_q3_2019.fillna(0)

# fix column names
#new_names = [replace(item) for item in list(sectors_q3_2019.columns)]
new_names = ['L_ACCOMMODATION_19', 'S_ADMIN_SUPPORT_19','A_AG_WILDLIFE_19','L_CULTURAL_REC_19','C_CONSTRUCTION_19',
'E_EDUCATION_19','S_FINANCIAL_19','L_RSTRNT_BAR_19','H_HEALTH_CARE_19','S_IT_DATA_19','S_CORPORATE_MGMT_19',
'M_MANUFACTURING_19','E_EXTRACTION_19','O_OTHER_19','X_ADJUSTMENTS_19','R_AUTO_PRIVATE_19','S_PROF_TECH_SRV_19',
'G_GOVERNMENT_19','S_RENTAL_LEASING_19','R_BUILDING_SUPPLY_19','R_CLOTHING_19','R_ELECTRONICS_19','R_GROCERY_BEV_19',
'R_FURNITURE_19','R_GAS_STATIONS_19','R_GENERAL_RETAIL_19','R_HEALTH_RETAIL_19','R_OTHER_RETAIL_19','R_AUTO_RETAIL_19',
'R_NONSTORE_RETAIL_19','R_SPORT_HOBBY_19','L_SPECIAL_EVENT_19','W_DISTRIBUTION_19','X_UNKNOWN_19','U_UTILITIES_19',
'W_WHLSALE_DURABLE_19','S_WHLSLE_ETRADE_19','W_WHLSLE_NDURABLE_19']
sectors_q3_2019.columns = new_names

# get the total sales tax revenue
sectors_q3_2019['TOTAL_19'] = sectors_q3_2019[new_names].sum(axis=1)

sectors_q3_2019.head()

### Calculate the differences 
*2020 - 2019 = Diff*

In [ ]:
# merge the 2019-2020 difference with the sdf
sectors_q3_diff = pd.DataFrame(sectors_q3_2020.values - sectors_q3_2019.values)

# add back the column names
new_names = ['L_ACCOMMODATION_D20', 'S_ADMIN_SUPPORT_D20','A_AG_WILDLIFE_D20','L_CULTURAL_REC_D20','C_CONSTRUCTION_D20',
'E_EDUCATION_D20','S_FINANCIAL_D20','L_RSTRNT_BAR_D20','H_HEALTH_CARE_D20','S_IT_DATA_D20','S_CORPORATE_MGMT_D20',
'M_MANUFACTURING_D20','E_EXTRACTION_D20','O_OTHER_D20','X_ADJUSTMENTS_D20','R_AUTO_PRIVATE_D20','S_PROF_TECH_SRV_D20',
'G_GOVERNMENT_D20','S_RENTAL_LEASING_D20','R_BUILDING_SUPPLY_D20','R_CLOTHING_D20','R_ELECTRONICS_D20','R_GROCERY_BEV_D20',
'R_FURNITURE_D20','R_GAS_STATIONS_D20','R_GENERAL_RETAIL_D20','R_HEALTH_RETAIL_D20','R_OTHER_RETAIL_D20','R_AUTO_RETAIL_D20',
'R_NONSTORE_RETAIL_D20','R_SPORT_HOBBY_D20','L_SPECIAL_EVENT_D20','W_DISTRIBUTION_D20','X_UNKNOWN_D20','U_UTILITIES_D20',
'W_WHLSALE_DURABLE_D20','S_WHLSLE_ETRADE_D20','W_WHLSLE_NDURABLE_D20', 'TOTAL_D20']

sectors_q3_diff.columns = new_names

# add back cities as index
sectors_q3_diff['City'] = sectors_q3_2020.index
sectors_q3_diff = sectors_q3_diff.set_index('City')

#check table
sectors_q3_diff.head(10)

In [ ]:
# Check join output shape
print(sectors_q3_2019.shape)
print(sectors_q3_2020.shape)
print(sectors_q3_diff.shape)

### Load municipalities and townships shapefile and join to formatted sales tax data

In [ ]:
# load cities shapefile into pandas spatial dataframe
cities_shp = '.\\Inputs\\Cities.shp'
cities_sdf = pd.DataFrame.spatial.from_featureclass(cities_shp)

# Rename columns
cities_sdf.columns = ['FID', 'NAME', 'SOURCE', 'SALESTAXID', 'POPLASTCEN', 'POPLASTEST',
       'AREA_SQMI', 'SHAPE']

In [ ]:
# merge the 2020 sdf with the pivoted data
cities_sdf2 = cities_sdf.merge(sectors_q3_2019, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q3_2020, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q3_diff, left_on= "NAME", right_on="City", how="inner")
cities_sdf2.tail(10)


In [ ]:
# Confirm join count, Salt Lake County (Unincorporated) should be the only one not joined
l1 = list(q3_2020['City'].value_counts().index)
l2 = list(cities_sdf['NAME'].value_counts().index)
list(set(l1) - set(l2))

In [ ]:
# Create file gdb and export sdf to feature class
outputs = '.\\Outputs'

gdb = os.path.join(outputs, "taxable_sales.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "taxable_sales.gdb")

cities_sdf2.spatial.to_featureclass(location=os.path.join(outputs, "taxable_sales.gdb","taxable_sales_utah_q3"))

### Analysis

In [ ]:
# Get year specific column names
standard_cols = ['fid','name','source','salestaxid','poplastcen','poplastcen','area_sqmi', 'SHAPE']
names_19 = standard_cols + [col for col in cities_sdf2.columns if '_19' in col]
names_20 = standard_cols + [col for col in cities_sdf2.columns if '_20' in col]
names_D20 = standard_cols + [col for col in cities_sdf2.columns if '_d20' in col]

# subset columns by time
cities_q3_19 = cities_sdf2[names_19]
cities_q3_20 = cities_sdf2[names_20]
cities_q3_d20 = cities_sdf2[names_D20]


In [ ]:
# 10 cities with smallest difference of revenue
cities_q3_d20[['name', 'poplastcen', 'area_sqmi', 'total_d20']].sort_values('total_d20', ascending=False).head(10)

In [ ]:
# 10 cities with largest difference of revenue
cities_q3_d20[['name', 'poplastcen', 'area_sqmi', 'total_d20']].sort_values('total_d20', ascending=True).head(10)

#### Some industries of interest:
- RETAIL-GASOLINE STATIONS (447)
- RETAIL-FOOD & BEVERAGE STORES (445)
- REAL ESTATE, RENTAL & LEASING (53)
- ARTS, ENTERTAINMENT AND RECREATION (71)
- ACCOMMODATION (721)